In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

In [ ]:
d="C:\\deha\\deha\\dataset\\titanic"
df_train=pd.read_csv(d+"\\train.csv")
df_test=pd.read_csv(d+"\\test.csv")

In [ ]:
df_train.columns

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
#Numerical Variables:PassengerID, Fare,Age
#Categorical Variables:Survived,Pclass,Ticket,Cabin,Embarked,Name,Sex,Sibsp,Parch

In [ ]:
def barplot(v):
    var=df_train[v]
    value=var.value_counts()
    plt.figure(figsize=(10,5))
    plt.bar(value.index,value,color="Pink")
    plt.title(v)
    plt.show()
    print(v,":\n",value)
col=["Sex","SibSp","Pclass","Embarked","Parch","Survived"]
for i in col:
    barplot(i)

In [ ]:
def histo(v):
    plt.figure(figsize=(9,3))
    plt.hist(df_train[v],bins=50)
    plt.xlabel(v)
    plt.ylabel("Frequency")
    plt.show()
numcol=["Age","Fare","PassengerId"]
for i in numcol:
    histo(i)

In [ ]:
#Basic Data Analysis

In [ ]:
#Pclass--Survived
df_train[["Pclass","Survived"]].groupby(["Pclass"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
#Sex--Survived
df_train[["Sex","Survived"]].groupby(["Sex"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
#SibSp--Survived
df_train[["SibSp","Survived"]].groupby(["SibSp"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
#Parch--Survived
df_train[["Parch","Survived"]].groupby(["Parch"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
#Outlier Detection

In [ ]:
def outlier(df_train,features):
    outlier_indices=[]
    for i in features:
        q1=np.percentile(df_train[i],25)
        q3=np.percentile(df_train[i],75)
        iqr=q3-q1
        step=iqr*1.5
        outliers=df_train[(df_train[i]<q1-step)|(df_train[i]>q3+step)].index
        outlier_indices.extend(outliers)
    outlier_indices=Counter(outlier_indices)
    mul_outliers=list(i for i,j in outlier_indices.items() if j>2)
    return mul_outliers
df_train.loc[outlier(df_train,["Age","Fare","SibSp","Parch"])]

In [ ]:
#drop outliers
df_train=df_train.drop(outlier(df_train,["Age","Fare","SibSp","Parch"]),axis=0).reset_index(drop=True)

In [ ]:
#Find missing values:

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train[df_train["Embarked"].isnull()]

In [ ]:
df_train.boxplot(column="Fare",by="Embarked")
plt.show()

In [ ]:
df_train["Embarked"]=df_train["Embarked"].fillna("C")
df_train[df_train["Embarked"].isnull()]

In [ ]:
df_train[df_train["Fare"].isnull()]
#df_train["Fare"] =df_train["Fare"].fillna(np.mean(df_train[df_train["Pclass"] == 3]["Fare"]))
#according to the data's pclass

In [ ]:
#visulaization

In [ ]:
#correlation

colist=["SibSp","Parch","Age","Fare","Survived"]
sns.heatmap(df_train[colist].corr(),annot=True)#fmt=".2f"
plt.show()

In [ ]:
#SibSp-----Survived

In [ ]:
sns.catplot(x="SibSp",y="Survived",data=df_train,kind="bar")
plt.show()

In [ ]:
sns.catplot(x="Pclass",y="Survived",data=df_train,kind="bar")
plt.show()

In [ ]:
com=sns.FacetGrid(df_train,col="Survived")
com.map(sns.distplot,"Age",bins=25)
plt.show()

In [ ]:
com=sns.FacetGrid(df_train,col="Survived",row="Pclass")
com.map(sns.histplot,"Age",bins=25)
plt.show()
#pclass is an important feature

In [ ]:
#Embarked--Sex--Pclass--Survived

In [ ]:
com=sns.FacetGrid(df_train,col="Embarked")
com.map(sns.pointplot,"Pclass","Survived","Sex")
com.add_legend()
plt.show()
#Embarked and Sex will be used in training

In [ ]:
#Embarked--Sex--Fare--Survived

In [ ]:
com=sns.FacetGrid(df_train,row="Embarked",col="Survived")
com.map(sns.barplot,"Sex","Fare")
com.add_legend()
plt.show()
#higher fare,higher survival rate ---- fare can be used as categorical for training

In [ ]:
#Fill Missing :Age Feature

In [ ]:
df_train[df_train["Age"].isnull()]

In [ ]:
#sns.catplot(x="Sex",y="Age",data=df_train,kind="box")
sns.boxplot(x="Sex",y="Age",data=df_train)
plt.show()
#Sex is not informative since it seems to be same for both male and female

In [ ]:
#sns.catplot(x="Sex",y="Age",hue="Pclass",data=df_train,kind="box")
sns.boxplot(x="Sex",y="Age",hue="Pclass",data=df_train)
plt.show()
#1 st class are older than 2nd and 2nd are older than 3rd.

In [ ]:
sns.catplot(x = "Parch", y = "Age", data = df_train, kind = "box")
sns.catplot(x = "SibSp", y = "Age", data = df_train, kind = "box")
plt.show()

In [ ]:
index_null=list(df_train["Age"][df_train["Age"].isnull()].index)
for i in index_null:
    age_pred = df_train["Age"][((df_train["SibSp"] == df_train.iloc[i]["SibSp"]) &(df_train["Parch"] == df_train.iloc[i]["Parch"])& (df_train["Pclass"] ==df_train.iloc[i]["Pclass"]))].median()
    age_med=df_train["Age"].median()
    if not np.isnan(age_pred):
        df_train["Age"].iloc[i]=age_pred
    else:
        df_train["Age"].iloc[i]=age_med

In [ ]:
df_train[df_train["Age"].isnull()]

In [ ]:
#Feature Engineering

In [ ]:
#Enter Title
name = df_train["Name"]
df_train["Title"] = [i.split(".")[0].split(",")[-1].strip() for i in name]

In [ ]:
df_train["Title"].head()

In [ ]:
sns.countplot(x="Title",data=df_train)
plt.xticks(rotation=65)
plt.show()

In [ ]:
#convert to categorical
df_train["Title"]=df_train["Title"].replace(["Lady","the Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"],"other")
df_train["Title"] = [0 if i == "Master" 
                     else 1 if i == "Miss" or i == "Ms" or i == "Mlle" or i == "Mrs" 
                     else 2 if i == "Mr" 
                     else 3 for i in df_train["Title"]]

In [ ]:
df_train["Title"].head()

In [ ]:
sns.countplot(x="Title",data=df_train)
plt.show()

In [ ]:
c=sns.catplot(x = "Title", y = "Survived", data = df_train, kind = "bar")
c.set_xticklabels(["Master","Mrs","Ms","Other"])
c.set_ylabels("Survival probability")
plt.show()

In [ ]:
#converting categorical as dummy variables
df_train.drop(labels=["Name"],axis=1,inplace=True)
df_train=pd.get_dummies(df_train,columns=["Title"])
df_train.head()

In [ ]:
#Enter Family Size
df_train["Famsize"]=df_train["SibSp"]+df_train["Parch"]+1
df_train.head()

In [ ]:
sns.countplot(x="Famsize",data=df_train)
plt.show()

In [ ]:
sns.catplot(x="Famsize",y="Survived",data=df_train,kind="bar")
plt.show()

In [ ]:
df_train["Famsize"]=[1 if i<5
                       else 0 for i in df_train["Famsize"]]
df_train.head()

In [ ]:
sns.catplot(x="Famsize",y="Survived",data=df_train,kind="bar")
plt.show()
#Small families have higher survival rate

In [ ]:
df_train=pd.get_dummies(df_train,columns=["Famsize"])
df_train.head()

In [ ]:
sns.countplot(x="Embarked",data=df_train)
plt.show()

In [ ]:
sns.catplot(x="Embarked",y="Survived",data=df_train,kind="bar")
plt.show()

In [ ]:
df_train = pd.get_dummies(df_train, columns=["Embarked"])
df_train.head()

In [ ]:
tickets=[]
for i in df_train["Ticket"]:
    if not i.isdigit():
        tickets.append(i.replace(".","").replace("/","").strip().split(" ")[0])
    else:
        tickets.append("x")
df_train["Ticket"]=tickets
df_train.head()

In [ ]:
df_train=pd.get_dummies(df_train,columns=["Ticket"],prefix="T")
df_train.head()

In [ ]:
#Pclass
sns.countplot(x="Pclass",data=df_train)
plt.show()

In [ ]:
df_train=pd.get_dummies(df_train,columns=["Pclass"])
df_train.head()

In [ ]:
df_train=pd.get_dummies(df_train,columns=["Sex"])
df_train.head()

In [ ]:
df_train.drop(labels=["Cabin"],axis=1,inplace=True)
df_train.columns

In [ ]:
#Modelling

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
x=int(len(df_train)*0.90)
print(x)

In [ ]:
test=df_train[x:]
test.drop(labels = ["Survived"],axis = 1, inplace = True)

In [ ]:
train = df_train[:x]
X_train=train.drop(labels="Survived",axis=1)
y_train=train["Survived"]
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.33,random_state=42)
print("X_train",len(X_train))
print("X_test",len(X_test))
print("y_train",len(y_train))
print("y_test",len(y_test))
print("test",len(test))

In [ ]:
X_train.head()

In [ ]:
#Logistic Regression

In [ ]:
logreg=LogisticRegression()
logreg.fit(X_train,y_train)
acc_log_train = round(logreg.score(X_train, y_train)*100,2) 
acc_log_test = round(logreg.score(X_test,y_test)*100,2)
print("Training Accuracy:",acc_log_train)
print("Testing Accuracy:",acc_log_test)

In [ ]:
random_state = 42
classifier = [DecisionTreeClassifier(random_state = random_state),
             SVC(random_state = random_state),
             RandomForestClassifier(random_state = random_state),
             LogisticRegression(random_state = random_state),
             KNeighborsClassifier()]

dt_param_grid = {"min_samples_split" : range(10,500,20),
                "max_depth": range(1,20,2)}

svc_param_grid = {"kernel" : ["rbf"],
                 "gamma": [0.001, 0.01, 0.1, 1],
                 "C": [1,10,50,100,200,300,1000]}
rf_param_grid = {"max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}

logreg_param_grid = {"C":np.logspace(-3,3,7),
                    "penalty": ["l1","l2"]}

knn_param_grid = {"n_neighbors": np.linspace(1,19,10, dtype = int).tolist(),
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}
classifier_param = [dt_param_grid,
                   svc_param_grid,
                   rf_param_grid,
                   logreg_param_grid,
                   knn_param_grid]

In [ ]:
cv_result = []
best_estimators = []
for i in range(len(classifier)):
    clf = GridSearchCV(classifier[i], param_grid=classifier_param[i], cv = StratifiedKFold(n_splits = 10), scoring = "accuracy", n_jobs = -1,verbose = 1)
    clf.fit(X_train,y_train)
    cv_result.append(clf.best_score_)
    best_estimators.append(clf.best_estimator_)
    print(cv_result[i])

In [ ]:
cv_results = pd.DataFrame({"Cross Validation Means":cv_result, "ML Models":["DecisionTreeClassifier", "SVM","RandomForestClassifier",
             "LogisticRegression",
             "KNeighborsClassifier"]})

g = sns.barplot(x="Cross Validation Means", y="ML Models", data = cv_results)
g.set_xlabel("Mean Accuracy")
g.set_title("Cross Validation Scores")

In [ ]:
votingC = VotingClassifier(estimators = [("dt",best_estimators[0]),
                                        ("rfc",best_estimators[2]),
                                        ("lr",best_estimators[3])],
                                        voting = "soft", n_jobs = -1)
votingC = votingC.fit(X_train, y_train)
print(accuracy_score(votingC.predict(X_test),y_test))

In [ ]:
test.head()

In [ ]:
clf = RandomForestClassifier(n_estimators = 100)
clf=clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_test),y_test))

In [ ]:
test_survived = pd.DataFrame(votingC.predict(test)).astype(int)
test_survived.columns=["Survived"]
test_survived.head()

In [ ]:
results = pd.DataFrame({"Passenger Id":test["PassengerId"],"Survived": test_survived})
results.to_csv("Final.csv", index = False)